# Grouped-Query Attention (GQA)

- 2023년에 제안된 [Grouped-Query Attention](https://arxiv.org/abs/2305.13245)은 최근 몇 년 동안 MHA보다 연산 및 parameter-efficient해서 MHA의 대안으로 자리잡았고, 근래에는 new standard가 되어버림.
  - 심지어 old Llama 2 시리즈도 이를 사용함.
- MHA에서는 각 head가 고유한 Key-Value set을 가졌지만, GQA는 메모리 사용량을 줄이기 위해 여러개의 head를 그룹화하여 동일한 Key-Value projection을 공유함.


![GQA](https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gqa-memory/1.webp)

- Key-Value 그룹이 3개이고, attention head가 6개라면, head 1과 head 2는 하나의 Key-Value set을 공유하고, head 3-4, head 5-6은 각각 다른 Key-Value set를 공유함.
- 이와같은 Key-Value sharing은 연산 횟수를 줄이고, 메모리 사용량을 줄이며 효율성을 증가시킴.
- 즉, 핵심 아이디어는 **여러 Query head에서 Key head와 Value head를 공유해서 Key-Value의 head 수를 줄이는 것**.
  - 모델의 parameter 수가 줄어들게 되고
  - Key-Value tensor에 대한 memory bandwidth usage(메모리 대역폭 사용량)을 줄여주는데, 이는 cache에 저장하고 검색해야 하는 Key-Value의 개수가 줄어들었기 때문.
- GQA는 주로 MHA의 계산 효율성을 높이기 위한 임시 방편이지만, (GQA paper와 Llama 2 paper에서 언급한 것 처럼) ablation study를 통해 LLM modeling 성능 측면에서 **standard MHA와 유사한 성능**을 보이는 것으로 나타남.


- 하지만, 이는 Key-Value 그룹의 개수를 신중하게 선택했다는 전제 하에 가능한 이야기.
  - 모든 attention head가 하나의 Key-Value 그룹을 공유하는 극단적인 경우, 메모리 사용량은 훨씬 더 감소하지만 modeling 성능은 저하될 수 있음.
  - 또한 Key-Value 그룹의 수를 Query의 head 수와 동일하게 설정하면 일반적인 MHA가 되어버림.

### Code examples

- 이전 `GPTModel`과 마찬가지로, 학습된 상태가 아니므로 의미없는 텍스트를 생성함.
- 앞서 작성했던 `kv-cache` 기능도 함꼐 사용함.

In [1]:
import time
import tiktoken
import torch
import torch.nn as nn

In [2]:
class GroupedQueryAttention(nn.Module):
    def __init__(self, d_in, d_out, dropout, num_heads, num_kv_groups, dtype=None, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
        assert num_heads % num_kv_groups == 0, "num_heads must be divisible by num_kv_groups"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        # 그룹이 head를 공유하므로 K, V의 차원은 num_kv_groups * head_dim
        self.W_key = nn.Linear(d_in, num_kv_groups * self.head_dim, bias=qkv_bias, dtype=dtype)
        self.W_value = nn.Linear(d_in, num_kv_groups * self.head_dim, bias=qkv_bias, dtype=dtype)
        self.num_kv_groups = num_kv_groups
        self.group_size = num_heads // num_kv_groups

        # Query는 head마다 다르므로 d_out 사용
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias, dtype=dtype)
        self.out_projection = nn.Linear(d_out, d_out, dtype=dtype)
        self.dropout = nn.Dropout(dropout)

        # KV-cache
        self.register_buffer('cache_k', None, persistent=False)
        self.register_buffer('cache_v', None, persistent=False)
        self.ptr_current_pos = 0

    def forward(self, x, use_cache=False):
        b, num_tokens, d_in = x.shape

        # Q, K, V projection
        Q = self.W_query(x)                     # [b, num_tokens, num_heads * head_dim]
        K, V = self.W_key(x), self.W_value(x)   # [b, num_tokens, num_kv_groups * head_dim]

        # Reshape -> 기존의 split() 기능
        Q = Q.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)          # [b, num_heads, num_tokens, head_dim]
        K_new = K.view(b, num_tokens, self.num_kv_groups, self.head_dim).transpose(1, 2)  # [b, num_kv_groups, n_tokens, head_dim]
        V_new = V.view(b, num_tokens, self.num_kv_groups, self.head_dim).transpose(1, 2)  # [b, num_kv_groups, n_tokens, head_dim]

        if use_cache:
            if self.cache_k is None:
                # cache에 이전 K, V가 없는 경우, 현재 K, V를 cache로 설정
                self.cache_k, self.cache_v = K_new, V_new
            else:
                # cache에 이전 K, V가 있는 경우, 현재 K, V를 이어붙임
                self.cache_k = torch.cat([self.cache_k, K_new], dim=2)  # n_tokens 차원(2번째 차원)을 따라 이어붙임
                self.cache_v = torch.cat([self.cache_v, V_new], dim=2)
            K_base, V_base = self.cache_k, self.cache_v
        else:
            K_base, V_base = K_new, V_new
            if self.cache_k is not None or self.cache_v is not None:
                self.cache_k, self.cache_v = None, None  # cache 초기화
                self.ptr_current_pos = 0
        
        # 각 head에 맞게 K, V 확장
            # tensor.repeat()와는 다르지만 numpy.repeat()와는 유사한 함수.
            # 원하는 row 별로 반복시켜서 tensor를 늘림.
        # shape는 [b, num_heads, num_tokens, head_dim]
        K = K_base.repeat_interleave(self.group_size, dim=1)
        V = V_base.repeat_interleave(self.group_size, dim=1)
        """
        # 예를 들어, dim=1에 맞춰 repat_interleave을 하기 전에 query groups의 형태는
        #   [K1, K2] 이고
        # repeat_interleave를 하게 되면 각 query group이 group_size 만큼 반복되어 
        #   [K1, K1, K2, K2] 가 됨
        # 일반적인 repeat 를 사용한다면 다음과 같은 형태의 tensor가 생성됨
        #   [K1, K2, K1, K2]
        """

        # Scaled Dot-Product Attention
        attn_scores = Q @ K.transpose(-2, -1)

        # Masking
        num_tokens_Q = Q.shape[-2]
        num_tokens_K = K.shape[-2]

        device = Q.device

        if use_cache:
            q_positions = torch.arange(
                self.ptr_current_pos,
                self.ptr_current_pos + num_tokens_Q,
                device=device,
                dtype=torch.long
            )
            self.ptr_current_pos += num_tokens_Q
        else:
            q_positions = torch.arange(num_tokens_Q, device=device, dtype=torch.long)
            self.ptr_current_pos = 0
        k_positions = torch.arange(num_tokens_K, device=device, dtype=torch.long)

        # 현재 Query 위치에 따라 masking을 어디까지 적용할지 boolean으로 지정
        mask = q_positions.unsqueeze(-1) < k_positions.unsqueeze(0)

        # 실제 Masking 수행
        attn_scores = attn_scores.masked_fill(mask, -torch.inf)

        # softmax scaling 및 dropout
        attn_weights = torch.softmax(attn_scores / K.shape[-1]**0.5, dim=-1)
        assert K.shape[-1] == self.head_dim, "head_dim mismatch"
        attn_weights = self.dropout(attn_weights)

        # (Q * K^T) * V 연산
            # [B, n_tokens, num_heads, d_head]
        context_vector = (attn_weights @ V).transpose(1, 2)

        # head들을 다시 CONCAT
            # [B, num_heads, n_tokens, d_head] -> [B, n_tokens, d_out]
        context_vector = context_vector.contiguous().view(b, num_tokens, self.d_out)
        context_vector = self.out_projection(context_vector)  # output projection

        return context_vector

    def reset_cache(self):
        self.cache_k, self.cache_v, = None, None
        self.ptr_current_pos = 0

In [3]:
from _04_gpt import LayerNorm, FeedForward

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.attention = GroupedQueryAttention(
            d_in = cfg['embed_dim'],
            d_out = cfg['embed_dim'],
            num_heads = cfg['num_heads'],
            num_kv_groups = cfg['num_kv_groups'],
            dropout = cfg['drop_rate'],
            qkv_bias = cfg['qkv_bias']
        )

        self.ffn = FeedForward(cfg)

        self.norm1 = LayerNorm(cfg['embed_dim'])
        self.norm2 = LayerNorm(cfg['embed_dim'])

        self.drop_shortcut = nn.Dropout(cfg['drop_rate'])

    def forward(self, x, use_cache=False):
        # attention with skip connection
        residual = x
        x = self.norm1(x)           # LayerNorm
        x = self.attention(x, use_cache=use_cache)       # MHA, [batch, context_length, embed_dim]

        x = self.drop_shortcut(x)   # Dropout
        x = x + residual            # skip(residual) connection

        # FFN with skip connection
        residual = x
        x = self.norm2(x)           # LayerNorm
        x = self.ffn(x)             # FeedForward
        x = self.drop_shortcut(x)   # Dropout
        x = x + residual            # skip(residual) connection

        return x

In [4]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.token_embedding = nn.Embedding(cfg['vocab_size'], cfg['embed_dim'])
        self.position_embedding = nn.Embedding(cfg['context_length'], cfg['embed_dim'])
        self.drop_embedding = nn.Dropout(cfg['drop_rate'])

        self.transformer_blocks = nn.ModuleList(
            [TransformerBlock(cfg) for _ in range(cfg['num_layers'])]
        )
        self.current_pos = 0

        self.final_norm = LayerNorm(cfg['embed_dim'])
        self.out_head = nn.Linear(cfg['embed_dim'], cfg['vocab_size'], bias=False)

    def forward(self, in_idx, use_cache=False):
        batch_size, seq_length = in_idx.shape

        # token embedding, positional embedding을 더해서 최종 input embedding 구성
        token_embeddings = self.token_embedding(in_idx)
        
        if use_cache:
            pos_ids = torch.arange(
                self.current_pos, self.current_pos + seq_length, device=in_idx.device, dtype=torch.long
            )
            self.current_pos += seq_length
        else:
            pos_ids = torch.arange(
                0, seq_length, device=in_idx.device, dtype=torch.long
            )
        pos_embeddings = self.position_embedding(pos_ids).unsqueeze(0)

        x = token_embeddings + pos_embeddings   # [batch_size, num_tokens, embed_dim]

        x = self.drop_embedding(x)

        # Transformer block forward pass
        for block in self.transformer_blocks:
            x = block(x, use_cache=use_cache)

        # last layer norm
        x = self.final_norm(x)

        logits = self.out_head(x)

        return logits
    
    def reset_kv_cache(self):
        for block in self.transformer_blocks:
            block.attention.reset_cache()
        self.current_pos = 0

In [6]:
from _04_gpt import generate_text_simple_cached

start_context = "O say can you see,"

tokenizer = tiktoken.get_encoding("gpt2")
encoded = tokenizer.encode(start_context)

GPT_CONFIG_124M = {
        'vocab_size': 50257,                    # Vocabulary size
        'context_length': 200 + len(encoded),   # Context(max sequence) length
        'embed_dim': 768,                       # Embedding dimension
        'num_heads': 12,                        # Number of attention heads
        'num_layers': 12,                       # Number of layers(transformer blocks)
        'drop_rate': 0.1,                       # Dropout rate
        'qkv_bias': False,                      # Q,K,V bias
        'num_kv_groups': 2                        # Number of KV groups for Grouped Query Attention
    }

torch.manual_seed(62)
model = GPTModel(GPT_CONFIG_124M)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

model.to(device)
model.eval()  # disable dropout

encoded_tensor = torch.tensor(encoded, device=device).unsqueeze(0)

print(f"\n{50*'='}\n{22*' '}IN\n{50*'='}")
print("\nInput text:", start_context)
print("Encoded input text:", encoded)
print("encoded_tensor.shape:", encoded_tensor.shape)

if torch.cuda.is_available():
    torch.cuda.synchronize()
start = time.time()

token_ids = generate_text_simple_cached(
    model = model,
    idx = encoded_tensor,
    max_new_tokens = 200
)

if torch.cuda.is_available():
    torch.cuda.synchronize()
total_time = time.time() - start

decoded_text = tokenizer.decode(token_ids.squeeze(0).tolist())
print(f"\n\n{50*'='}\n{22*' '}OUT\n{50*'='}")
print("\nOutput:", token_ids)
print("Output length:", len(token_ids[0]))
print("Output text:", decoded_text)

print(f"\nTime: {total_time:.2f} sec")
print(f"{int(len(token_ids[0])/total_time)} tokens/sec")

if torch.cuda.is_available():
    max_mem_bytes = torch.cuda.max_memory_allocated()
    max_mem_gb = max_mem_bytes / (1024 ** 3)
    print(f"Max memory allocated: {max_mem_gb:.2f} GB")

Using device: cuda

                      IN

Input text: O say can you see,
Encoded input text: [46, 910, 460, 345, 766, 11]
encoded_tensor.shape: torch.Size([1, 6])


                      OUT

Output: tensor([[   46,   910,   460,   345,   766,    11,  7232,  1317, 31438, 44667,
         11350, 27074, 43236, 26816, 33203, 40790, 33220, 42879, 45069, 30262,
         39066, 35938,   673, 18768,  4804, 33561,  6951,  4270, 13295, 20613,
            63, 28113, 30447, 43365,  6328, 14683, 11419, 21948, 22109, 16193,
         20573, 19869, 21658,   924, 44084, 16696,  5719, 39643,   294, 15099,
          5017, 22501, 39615, 21103, 48363, 27820, 24539, 25824,  1309, 23529,
         14793,  7261, 25882, 40040, 12507, 40393, 17816, 28792, 10225, 17515,
         16107, 44952, 40800,  9787, 22398, 19091, 47068,  3206, 19586, 35222,
         23009, 17474, 37040, 32909, 10380, 23172, 23814, 27907, 31103,  8349,
         48374, 31443, 46595, 23920, 48509,  7784,  8643, 33366, 23585, 35500,
      

In [7]:
from _04_gpt import main
main()

Using device: cuda

                      IN

Input text: Hello, I am
Encoded input text: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


                      OUT

Output: tensor([[15496,    11,   314,   716, 38718, 11139,  4535, 46798, 39622, 20124,
          9799,  1330, 13403, 14447,  4748, 30387, 43330, 19030, 33606,  9908,
         18323, 29347, 36465, 46135,  3177, 41268, 49233,  5027, 44525, 28370,
         18210, 38214, 33964, 29150, 38376, 23223, 34025, 33589, 10189, 26951,
          1404, 10914,  5383, 17720, 43639,  5607, 46954, 15773, 29286, 13582,
         44134, 12551, 38363,  9995, 12420,  4536, 19795, 33606, 35105, 32756,
         41971, 48907, 22706, 32522, 32289, 24683, 22067, 29461, 13228,  8932,
          5536,  6072, 14258, 39508, 32757, 10243, 13411, 40950, 13869, 19253,
          7202, 35644,  2297,  4533,  5171, 39923,  4785,  2770,  4442, 24681,
         26380, 26702,  6744, 24616, 42528, 48837,  5998, 25043, 11646,  5684,
         29422, 4402

- 마찬가지로, KV-cache를 사용하지 않은 pure MHA보다 초당 처리하는 토큰 수는 많아졌음.
- 또한, Key-Value의 head 수가 명시적으로 줄어들어서 (grouped 연산) 사용하는 GPU memory 가 훨씬 줄어들었음.  <br>
  (0.58 GB vs. 1.31 GB)
- 그럼에도 불구하고 용량을 꽤나(?) 차지하는 이유는, 모델의 FFN layer가 대부분을 차지하기 때문.


[GQA(이외에 MLA, DSA) 참고 유튜브 링크 1](https://youtu.be/Y-o545eYjXM)  <br>
[GQA(이외에 MQA) 참고 유튜브 링크 2](https://youtu.be/pVP0bu8QA2w)